In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [8]:
# load link counts data
link_day_Jan_dict = zload('../temp_files/link_day_Jan_dict.pkz')
link_day_Apr_dict = zload('../temp_files/link_day_Apr_dict.pkz')
link_day_Jul_dict = zload('../temp_files/link_day_Jul_dict.pkz')
link_day_Oct_dict = zload('../temp_files/link_day_Oct_dict.pkz')

In [9]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [10]:
K = len(week_day_Jan_list) + len(week_day_Apr_list) + len(week_day_Jul_list) + len(week_day_Oct_list)
#K = len(week_day_Jan_list) + len(week_day_Apr_list)

In [11]:
K

88

In [12]:
x = np.zeros((24, K))

In [14]:
m = 24

for k_1 in range(len(week_day_Jan_list)):
    for i in range(m):
        x[i, k_1] = link_day_Jan_dict['link_' + str(i) + '_' + str(week_day_Jan_list[k_1])].AM_flow

for k_2 in range(len(week_day_Apr_list)):
    for i in range(m):
        x[i, k_2 + k_1 + 1] = link_day_Apr_dict['link_' + str(i) + '_' + str(week_day_Apr_list[k_2])].AM_flow

for k_3 in range(len(week_day_Jul_list)):
    for i in range(m):
        x[i, k_3 + k_2 + k_1 + 2] = link_day_Jul_dict['link_' + str(i) + '_' + str(week_day_Jul_list[k_3])].AM_flow
for k_4 in range(len(week_day_Oct_list)):
    for i in range(m):
        x[i, k_4 + k_3 + k_2 + k_1 + 3] = link_day_Oct_dict['link_' + str(i) + '_' + str(week_day_Oct_list[k_4])].AM_flow

x = np.matrix(x)
x = np.nan_to_num(x)

In [15]:
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [16]:
np.size(x, 1)

86

In [17]:
link_day_Jan_dict['link_2_2'].AM_flow, link_day_Jan_dict['link_2_1'].AM_flow

(639708.01714969077, nan)

In [18]:
np.size(x,0), np.size(x,1)

(24, 86)

In [19]:
np.size(A,0), np.size(A,1)

(24, 140)

In [20]:
# calculate the GLS solution for the O-D matrix

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

term_1 = inv(np.dot(np.dot(PA_t, inv_S), AP_t))
term_2 = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])

lam_est = (1.0 / K) * np.dot(term_1, term_2)

In [34]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())




Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-02, 2e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.60570405e+06  0.00000000e+00  5.90e+03 3.76e+02  9.16e+05     0s
   1   5.43308970e+05 -2.03615380e+06  1.46e+03 2.46e+02  2.78e+05     0s
   2   4.91740417e+05 -1.70978661e+06  1.46e-03 5.68e-14  3.93e+04     0s
   3  -2.56129778e+05 -5.55418309e+05  1.46e-09 5.68e-14  5.3

In [42]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [35]:
lam_list

[1.28326568563071e-08,
 132317.91565476044,
 121387.8283282293,
 8.940064445506907e-06,
 63826.04864740836,
 1.110979162233152e-05,
 2.0481892994238596e-05,
 120303.68969267681,
 7.943322558403066e-10,
 91544.5780912974,
 7.92374306922208e-10,
 7.92441286601813e-10,
 7.923859643541755e-10,
 7.996545144069961e-10,
 1.5639214041170656e-06,
 3.484458713151877e-06,
 1.5389040798815265e-06,
 8.805674189602333e-06,
 182384.98623645608,
 279434.88496791804,
 1.8889633990316207e-05,
 1.5412065953410506e-05,
 237834.24677651213,
 85402.93464939862,
 5.798327957556859e-05,
 5.611326714059783e-05,
 6.973199747743047e-05,
 120195.11691761157,
 183507.05843415068,
 54878.942436623394,
 195688.70937543458,
 35254.639442121996,
 92344.73156256636,
 1.0798748992073021e-06,
 80908.97046726948,
 4.4134321307226845e-06,
 1.3197503569574442e-05,
 347834.1919611446,
 7.0831725557177205e-06,
 84395.66602366693,
 170777.82214787893,
 6570.220648076328,
 6.906847588415569e-07,
 3.9707094747210014e-06,
 2.4758

In [353]:
x[:,3]

matrix([[ 336090.2248882 ],
        [ 347781.98764409],
        [ 623611.08100628],
        [ 646565.47043538],
        [ 167366.58140388],
        [ 222199.77728294],
        [ 395021.98366766],
        [ 404603.81413865],
        [ 391869.05511341],
        [ 407930.2842203 ],
        [ 390596.28051459],
        [ 360706.68193882],
        [  99054.76534118],
        [  97082.30249335],
        [ 172763.26094238],
        [ 146287.14855371],
        [ 214727.44269757],
        [ 210173.2555332 ],
        [ 338221.96584561],
        [ 348849.27868725],
        [ 488463.61641117],
        [ 557490.77861642],
        [ 136489.90091001],
        [ 139044.61460856]])

In [320]:
len(x[:,1])

24

In [321]:
K

88